In [96]:
import weaviate, os

In [85]:
client.close()

In [28]:
pip install -U weaviate-client


[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [109]:
weaviate.__version__

'4.6.5'

In [110]:
import weaviate
import os
import requests
import json
import weaviate.classes.config as wc

# AWS Credentials
AWS_ACCESS_KEY_ID = os.getenv('AWS_BEDROCK_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_BEDROCK_SECRET_ACCESS_KEY')
AWS_SESSION_TOKEN = os.getenv('AWS_BEDROCK_SESSION_TOKEN')

# Bedrock Endpoint
BEDROCK_ENDPOINT = 'https://bedrock.amazonaws.com'

# Weaviate Connection

client = weaviate.connect_to_wcs(
     cluster_url='https://sadiq-v2-qoo9zcnn.weaviate.network',                             # Replace with your WCS cluster ID
     auth_credentials=weaviate.auth.AuthApiKey('8L8evblmIuJi3fhc9Iut5W6k8jEar10E2fda'), # Replace with your WCS API KEY - recommended: use env var
      headers={
          "X-AWS-Access-Key": AWS_BEDROCK_ACCESS_KEY_ID, # Replace with your AWS access key - recommended: use env var
          "X-AWS-Secret-Key": AWS_BEDROCK_SECRET_ACCESS_KEY, # Replace with your AWS secret key - recommended: use env var
          #"X-AWS-Session-Key": AWS_BEDROCK_SESSION_TOKEN

        },
    #timeout_config=weaviate.util.Config.Timeout(init=60),  # Increase init timeout to 60 seconds
    #startup_config=weaviate.util.Config.Startup(skip_init_checks=True)  # Skip startup checks
)

print(f'Clinet Ready  with connect : {client.is_ready()}')

# Check if Weaviate is ready
if not client.is_ready():
    raise Exception("Weaviate client is not ready")



# Delete the collection if it already exists
if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyQuestion",

    vectorizer_config=wc.Configure.Vectorizer.text2vec_aws(
        model="cohere.embed-english-v3", # select the model, make sure it is enabled for your account
        service="bedrock",
        #source_properties=["title"],
        region="us-east-1"               # select your region
    ),

    properties=[ # defining properties (data schema) is optional
        wc.Property(name="Question", data_type=wc.DataType.TEXT), 
        wc.Property(name="Answer", data_type=wc.DataType.TEXT),
        wc.Property(name="Category", data_type=wc.DataType.TEXT, skip_vectorization=True), 
    ]
)

print("Successfully created collection: JeopardyQuestion.")

# Fetch the Jeopardy dataset
url = "https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json"
resp = requests.get(url)
data = json.loads(resp.text)

collection = client.collections.get("JeopardyQuestion")

with collection.batch.dynamic() as batch:
    for src_obj in data:
        weaviate_obj = {
            "Question": src_obj["Question"],
            "Answer": src_obj["Answer"],
            "Category": src_obj["Category"],
        }

        # The model provider integration will automatically vectorize the object
        batch.add_object(
            properties=weaviate_obj,
            # vector=vector  # Optionally provide a pre-obtained vector
        )


Clinet Ready  with connect : True
Successfully created collection: JeopardyQuestion.


In [111]:
print(data[0])
print(data[1])

{'Category': 'SCIENCE', 'Question': 'This organ removes excess glucose from the blood & stores it as glycogen', 'Answer': 'Liver'}
{'Category': 'ANIMALS', 'Question': "It's the only living mammal in the order Proboseidea", 'Answer': 'Elephant'}


In [112]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects()

for o in response.objects:
    print(o.properties)

In [113]:
#collection = client.collections.get("JeopardyQuestion")

response = collection.query.hybrid(
    query="This organ removes excess glucose from the blood & stores it as glycogen",  
    # The model provider integration will automatically vectorize the query
    limit=2
)

print(response)

for obj in response.objects:
    print(obj.properties["Answer"])
    


WeaviateQueryError: Query call with protocol GRPC search failed with message remote client vectorize: Couldn't invoke cohere.embed-english-v3 model: operation error Bedrock Runtime: InvokeModel, https response error StatusCode: 403, RequestID: 73fc7ce4-326f-4d0c-88c0-d933537f71a6, api error UnrecognizedClientException: The security token included in the request is invalid..

In [114]:
from weaviate.classes.query import MetadataQuery

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects()

print(response)

response = jeopardy.query.near_text(
    query="This organ removes excess glucose from the blood & stores it as glycogen",
    limit=2,
    return_metadata=MetadataQuery(distance=True)
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.distance)

QueryReturn(objects=[])


WeaviateQueryError: Query call with protocol GRPC search failed with message explorer: get class: vectorize params: vectorize params: vectorize params: vectorize keywords: remote client vectorize: Couldn't invoke cohere.embed-english-v3 model: operation error Bedrock Runtime: InvokeModel, https response error StatusCode: 403, RequestID: 8fb4d220-9447-4210-88c2-28d2bb51c334, api error UnrecognizedClientException: The security token included in the request is invalid..